In [12]:
%pip install lightgbm==3.3.2
%pip install catboost==0.26.1

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 4.0 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.6.0
    Uninstalling lightgbm-4.6.0:
      Successfully uninstalled lightgbm-4.6.0
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 0.1.1
ERROR: Could not find a version that satisfies the requirement catboost==0.26.1 (from versions: 1.2, 1.2.1, 1.2.2, 1.2.3, 1.2.5, 1.2.6, 1.2.7, 1.2.8)

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for catboost==0.26.1


In [3]:
# =============================
# 📌 Cell 1: Install & Imports
# =============================


import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")


In [4]:
# =============================
# 📌 Cell 2: Load Data
# =============================
train = pd.read_csv("C:\\Users\\PC\\Desktop\\Bootcamp week 1 task\\BOOTCAMP-PROJECT\\giki-solar-energy-prediction-challenge\\train_data.csv")
test = pd.read_csv("C:\\Users\\PC\\Desktop\\Bootcamp week 1 task\\BOOTCAMP-PROJECT\\giki-solar-energy-prediction-challenge\\test_data_masked.csv")
systems = pd.read_csv("C:\\Users\\PC\\Desktop\\Bootcamp week 1 task\\BOOTCAMP-PROJECT\\giki-solar-energy-prediction-challenge\\systems_new.csv")

print(train.head())
print(test.head())
print(systems.head())


   system_id            timestamp  generation_W     load_W
0         17  2023-08-14 01:50:00           0.0  525.66667
1         17  2023-08-14 02:00:00           0.0  518.90000
2         17  2023-08-14 02:10:00           0.0  528.50001
3         17  2023-08-14 02:20:00           0.0  517.99999
4         17  2023-08-14 02:30:00           0.0  520.90001
    test_id  system_id            timestamp  generation_W      load_W
0  QNN8B4SF         39  2023-08-12 17:40:00    2770.28574  3410.08823
1  2XFLJU7J         39  2023-08-12 17:50:00    2198.08336  3183.79486
2  ZTXR24I5         39  2023-08-12 18:00:00    1801.88889  2743.21568
3  AE6LFBMV         39  2023-08-12 18:10:00    1419.09090  1630.29630
4  RSDMSDW2         39  2023-08-12 18:20:00     897.99999  1788.60000
   system_id connection_type     location  panels_capacity  load_capacity
0          1     RESIDENTIAL    ISLAMABAD           11.125           10.0
1          2      COMMERCIAL  SHEIKHUPURA            5.340            5.5
2   

In [5]:
# =============================
# 📌 Cell 3: Merge Metadata
# =============================
train = train.merge(systems, on="system_id", how="left")
test = test.merge(systems, on="system_id", how="left")


In [6]:
# =============================
# 📌 Cell 4: Feature Engineering
# =============================
def create_features(df):
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["hour"] = df["timestamp"].dt.hour
    df["minute"] = df["timestamp"].dt.minute
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    df["month"] = df["timestamp"].dt.month
    
    # Lags & Rolling
    for col in ["generation_W", "load_W"]:
        if col in df.columns:
            for lag in [1, 2, 3, 6]:
                df[f"{col}_lag{lag}"] = df.groupby("system_id")[col].shift(lag)
            df[f"{col}_rolling3"] = df.groupby("system_id")[col].shift(1).rolling(3).mean()
            df[f"{col}_rolling6"] = df.groupby("system_id")[col].shift(1).rolling(6).mean()
    
    return df

train = create_features(train)
test = create_features(test)


In [7]:
# =============================
# 📌 Cell 5: Encode Categorical
# =============================
for col in ["connection_type", "location"]:
    le = LabelEncoder()
    full = pd.concat([train[col], test[col]], axis=0).astype(str)
    le.fit(full)
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))


In [8]:
# =============================
# 📌 Cell 6: Define Features & Targets
# =============================
target_cols = ["generation_W", "load_W"]
drop_cols = ["timestamp"] + target_cols

features = [col for col in train.columns if col not in drop_cols]
print("Using features:", features)


Using features: ['system_id', 'connection_type', 'location', 'panels_capacity', 'load_capacity', 'hour', 'minute', 'dayofweek', 'month', 'generation_W_lag1', 'generation_W_lag2', 'generation_W_lag3', 'generation_W_lag6', 'generation_W_rolling3', 'generation_W_rolling6', 'load_W_lag1', 'load_W_lag2', 'load_W_lag3', 'load_W_lag6', 'load_W_rolling3', 'load_W_rolling6']


In [ ]:
# =============================
# 📌 Cell 7: Train Base Models (Stacking, CUDA Compatible)
# =============================
oof_preds = {t: np.zeros(len(train)) for t in target_cols}
test_preds = {t: np.zeros(len(test)) for t in target_cols}

kf = GroupKFold(n_splits=5)

for target in target_cols:
    print(f"\nTraining models for {target}")
    X = train[features]
    y = train[target]
    groups = train["system_id"]
    Xt = test[features]
    
    oof = np.zeros(len(train))
    fold_preds = np.zeros(len(test))
    
    for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y, groups)):
        X_tr, y_tr = X.iloc[trn_idx], y.iloc[trn_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        # ⚡ LightGBM (GPU)
        lgb_model = lgb.LGBMRegressor(
            n_estimators=500,
            learning_rate=0.05,
            device="gpu"  # enable GPU
        )
        lgb_model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="mae"
        )
        lgb_val = lgb_model.predict(X_val)
        lgb_test = lgb_model.predict(Xt)
        
        # ⚡ XGBoost (GPU)
        xgb_model = xgb.XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            tree_method="gpu_hist",  # use GPU
            predictor="gpu_predictor"
        )
        xgb_model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        xgb_val = xgb_model.predict(X_val)
        xgb_test = xgb_model.predict(Xt)
        
        # ⚡ CatBoost (GPU)
        cb_model = cb.CatBoostRegressor(
            iterations=500,
            learning_rate=0.05,
            task_type="GPU",   # use GPU
            devices="0",       # first GPU
            verbose=False
        )
        cb_model.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True)
        cb_val = cb_model.predict(X_val)
        cb_test = cb_model.predict(Xt)
        
        # ElasticNet (CPU only)
        from sklearn.impute import SimpleImputer

# Imputer for ElasticNet (mean strategy for numerical features)
        imputer = SimpleImputer(strategy="mean")

# Inside your training loop, replace ElasticNet section:
# ------------------------------------------------------
# Impute missing values
        X_tr_enet = imputer.fit_transform(X_tr)
        X_val_enet = imputer.transform(X_val)
        Xt_enet = imputer.transform(Xt)

# ElasticNet (CPU only)
        enet = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)
        enet.fit(X_tr_enet, y_tr)
        enet_val = enet.predict(X_val_enet)
        enet_test = enet.predict(Xt_enet)

        
        # Stack predictions
        stack_val = np.vstack([lgb_val, xgb_val, cb_val, enet_val]).T
        stack_test = np.vstack([lgb_test, xgb_test, cb_test, enet_test]).T
        
        # Meta model (Ridge, CPU)
        meta = Ridge(alpha=1.0)
        meta.fit(stack_val, y_val)
        
        oof[val_idx] = meta.predict(stack_val)
        fold_preds += meta.predict(stack_test) / kf.n_splits
    
    oof_preds[target] = oof
    test_preds[target] = fold_preds
    
    print(f"{target} OOF MAE:", mean_absolute_error(train[target], oof))



Training models for generation_W
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3244
[LightGBM] [Info] Number of data points in the train set: 3079330, number of used features: 21
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 960, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 21 dense feature groups (70.48 MB) transferred to GPU in 0.107674 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 1810.037046


In [ ]:
# =============================
# 📌 Cell 8: Create Submission
# =============================
submission = test[["system_id", "timestamp"]].copy()
submission["generation_W"] = test_preds["generation_W"]
submission["load_W"] = test_preds["load_W"]

submission.to_csv("submission.csv", index=False)
print("✅ Submission file created!")
